In [ ]:
import pandas as pd
import numpy as np
import os
import time
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc
from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import Normalizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from joblib import dump, load


In [ ]:
def load_data(data_path, data_path_time):
    time_start=time.time()
    
    #Load data
    data_list = []
    for fname in sorted(os.listdir(data_path)):
        subject_data_path = os.path.join(data_path, fname)
        print(subject_data_path)
        if not os.path.isfile(subject_data_path): continue
        data_list.append(
            pd.read_csv(
                        subject_data_path,
                        sep='|', 
                        header=None,
                        names = [
                                'CREDIT_SCORE',
                                'FIRST_PAYMENT_DATE',
                                'FIRST_TIME_HOMEBUYER_FLAG',
                                '4','5','6',
                                'NUMBER_OF_UNITS',
                                'OCCUPANCY_STATUS',
                                '9',
                                'ORIGINAL_DTI_RATIO',
                                'ORIGINAL_UPB',
                                'ORIGINAL_LTV',
                                'ORIGINAL_INTEREST_RATE',
                                'CHANNEL',
                                '15',
                                'PRODUCT_TYPE',
                                'PROPERTY_STATE',
                                'PROPERTY_TYPE',
                                '19',
                                'LOAN_SQ_NUMBER',
                                'LOAN_PURPOSE',
                                'ORIGINAL_LOAN_TERM',
                                'NUMBER_OF_BORROWERS',
                                '24','25','26','27'#data from every year may have different column number
                            #2004-2007: 27 2008: 26 2009: 27
                        ],
                        usecols=[
                            'CREDIT_SCORE',
                            'FIRST_TIME_HOMEBUYER_FLAG',
                            'NUMBER_OF_UNITS',
                            'OCCUPANCY_STATUS',
                            'ORIGINAL_DTI_RATIO',
                            'ORIGINAL_UPB',
                            'ORIGINAL_LTV',
                            'ORIGINAL_INTEREST_RATE',
                            'CHANNEL',
                            'PROPERTY_TYPE',
                            'LOAN_SQ_NUMBER',
                            'LOAN_PURPOSE',
                            'ORIGINAL_LOAN_TERM',
                            'NUMBER_OF_BORROWERS'
                        ],
                        dtype={'CREDIT_SCORE':np.float_, 
                               'FIRST_TIME_HOMEBUYER_FLAG':np.str, 
                               'NUMBER_OF_UNITS':np.int_, 
                               'OCCUPANCY_STATUS':np.str,
                               'ORIGINAL_DTI_RATIO':np.float_,
                               'ORIGINAL_UPB':np.float_,
                               'ORIGINAL_LTV':np.float_,
                               'ORIGINAL_INTEREST_RATE':np.float_,
                               'CHANNEL':np.str,
                               'PROPERTY_TYPE':np.str,
                               'LOAN_SQ_NUMBER':np.str,
                               'LOAN_PURPOSE':np.str,
                               'ORIGINAL_LOAN_TERM':np.int_,
                               'NUMBER_OF_BORROWERS':np.int_},
                        low_memory=False
                        )
        )
    data = pd.concat(data_list)
    
    #Load data with time
    data_p_list=[]
    for fname in sorted(os.listdir(data_path_time)):
        subject_data_path = os.path.join(data_path_time, fname)
        print(subject_data_path)
        if not os.path.isfile(subject_data_path): continue
        data_p_list.append(
            pd.read_csv(subject_data_path,
                             sep='|',
                             header=None,
                             usecols=[0,3,4],
                             dtype={'0':np.str, '3':np.str, '4':np.int_}
                            )
        )
    #data_p = pd.concat(data_p_list)
    
    #Calculate default
    default_list=[]
    for data_p in data_p_list:
        data_p[3] = data_p[3].astype(str)
        clean_index = data_p.iloc[:,1].str.isdigit()
        data_p_cleaned = data_p[clean_index].copy()
        data_p_cleaned[3] = data_p_cleaned[3].astype(int)
        data_less_than_48 = data_p_cleaned[data_p_cleaned[4] < 48]
        default_list.append(data_less_than_48[data_less_than_48[3] > 2])
    
    data_default = pd.concat(default_list)
    
    default_index = data['LOAN_SQ_NUMBER'].isin(data_default[0].tolist())
    
    
    
    data['default_flag']=default_index
    
    
    data.drop(columns=['LOAN_SQ_NUMBER'], inplace=True)
    #data.to_csv('data/historical_data_withflag.csv',index=False)
    
    #Imputation
    CREDIT_SCORE = data['CREDIT_SCORE']
    OIR = data['ORIGINAL_DTI_RATIO']
    LTV = data['ORIGINAL_LTV']
    CREDIT_clean = CREDIT_SCORE[CREDIT_SCORE != 9999]
    OIR_clean = OIR[OIR != 999]
    LTV_clean = LTV[LTV != 999]
    data['CREDIT_SCORE'] = data['CREDIT_SCORE'].apply(lambda x : CREDIT_clean.mean() if x == 9999 else x)
    data['ORIGINAL_DTI_RATIO'] = data['ORIGINAL_DTI_RATIO'].apply(lambda x : OIR_clean.mean() if x == 999 else x)
    data['ORIGINAL_LTV'] = data['ORIGINAL_LTV'].apply(lambda x : LTV_clean.mean() if x == 999 else x)
    
    
    
    #Timer stop
    time_end=time.time()
    print('Finished loading, time cost:',time_end-time_start,'s')
    return data

In [ ]:
def train_log(data, fig, ax, year):
   
    time_start=time.time()
    #Get dummy value
    
    data['FIRST_TIME_HOMEBUYER_FLAG'] = data['FIRST_TIME_HOMEBUYER_FLAG'].apply(lambda x : np.NaN if x == '9' else x)
    data['NUMBER_OF_UNITS'] = data['NUMBER_OF_UNITS'].apply(lambda x : np.NaN if x == 99 else x)
    data['CHANNEL'] = data['CHANNEL'].apply(lambda x : np.NaN if x == 'T' else x)
    data['PROPERTY_TYPE'] = data['PROPERTY_TYPE'].apply(lambda x : np.NaN if x == '99' else x)
    data['NUMBER_OF_BORROWERS'] = data['NUMBER_OF_BORROWERS'].apply(lambda x : np.NaN if x == 99 else x)
    
    
    output_array = np.asarray(data['default_flag'].astype(int))
    input_array = np.c_[
        data[['CREDIT_SCORE',#0
              'ORIGINAL_DTI_RATIO',#1
              'ORIGINAL_UPB',#2
              'ORIGINAL_LTV',#3
              'ORIGINAL_LOAN_TERM',#4
              'ORIGINAL_INTEREST_RATE'#5
             ]],
        np.asarray(pd.get_dummies(data['FIRST_TIME_HOMEBUYER_FLAG'])), # N,Y,9 str remove 9
        np.asarray(pd.get_dummies(data['NUMBER_OF_UNITS'])), # 1 2 3 4 99 int remove 99
        np.asarray(pd.get_dummies(data['OCCUPANCY_STATUS'])), # P S I str
        np.asarray(pd.get_dummies(data['CHANNEL'])), # T R C B str remove T
        np.asarray(pd.get_dummies(data['PROPERTY_TYPE'])), # SF PU CO MH CP 99 str remove 99
        np.asarray(pd.get_dummies(data['LOAN_PURPOSE'])), # P N C str
        np.asarray(pd.get_dummies(data['NUMBER_OF_BORROWERS'])) # 2 1 99 int remove 99
    ]
    
    #Normalise
    #min_max_scaler = preprocessing.MinMaxScaler()
    #input_array_N = min_max_scaler.fit_transform(input_array)
    
#     train_path_X = 'datasets/train/features/'
#     train_path_y = 'datasets/train/target/'
#     test_path_X = 'datasets/test/features/'
#     test_path_y = 'datasets/test/target/'
    
    
    X = input_array
    y = output_array
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.3, 
        random_state=13
    )
    
    classifier = LogisticRegression(
        tol= 1e-5,
        #C=0.1,
        #class_weight = 'balanced',
        #class_weight = {0:0.01, 1:0.99},
        #solver='sag',
        max_iter=1500,
        n_jobs = -1
    )
    
    classifier.fit(X_train,y_train)
    
    viz = plot_roc_curve(
        classifier, 
        X_test, 
        y_test,
        name='Test ROC'.format(0),
        alpha=0.5, lw=1, ax=ax
    )
    
    viz_train = plot_roc_curve(
        classifier, 
        X_train, 
        y_train,
        name='Train ROC'.format(1),
        alpha=0.5, lw=1, ax=ax
    ) 
    
    ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
        label='Chance', alpha=.8)
    
    ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
       title="Receiver operating characteristic example")
    ax.legend(loc="lower right")
    time_end=time.time()
    print('Training done, time cost:',time_end-time_start,'s')
    
    print(classifier.predict_proba(X_test)[:, 1])
    return classifier, X_test, y_test, X_train, y_train

In [ ]:
def train_NN(data, fig, ax):
   
    time_start=time.time()
    #Get dummy value
    
    data['FIRST_TIME_HOMEBUYER_FLAG'] = data['FIRST_TIME_HOMEBUYER_FLAG'].apply(lambda x : np.NaN if x == '9' else x)
    data['NUMBER_OF_UNITS'] = data['NUMBER_OF_UNITS'].apply(lambda x : np.NaN if x == 99 else x)
    data['CHANNEL'] = data['CHANNEL'].apply(lambda x : np.NaN if x == 'T' else x)
    data['PROPERTY_TYPE'] = data['PROPERTY_TYPE'].apply(lambda x : np.NaN if x == '99' else x)
    data['NUMBER_OF_BORROWERS'] = data['NUMBER_OF_BORROWERS'].apply(lambda x : np.NaN if x == 99 else x)
    
    
    output_array = np.asarray(data['default_flag'].astype(int))
    input_array = np.c_[
        data[['CREDIT_SCORE',#0
              'ORIGINAL_DTI_RATIO',#1
              'ORIGINAL_UPB',#2
              'ORIGINAL_LTV',#3
              'ORIGINAL_LOAN_TERM',#4
              'ORIGINAL_INTEREST_RATE'#5
             ]],
        np.asarray(pd.get_dummies(data['FIRST_TIME_HOMEBUYER_FLAG'])), # N,Y,9 str remove 9
        np.asarray(pd.get_dummies(data['NUMBER_OF_UNITS'])), # 1 2 3 4 99 int remove 99
        np.asarray(pd.get_dummies(data['OCCUPANCY_STATUS'])), # P S I str
        np.asarray(pd.get_dummies(data['CHANNEL'])), # T R C B str remove T
        np.asarray(pd.get_dummies(data['PROPERTY_TYPE'])), # SF PU CO MH CP 99 str remove 99
        np.asarray(pd.get_dummies(data['LOAN_PURPOSE'])), # P N C str
        np.asarray(pd.get_dummies(data['NUMBER_OF_BORROWERS'])) # 2 1 99 int remove 99
    ]
    
    #Normalise
    min_max_scaler = preprocessing.MinMaxScaler()
    input_array_N = min_max_scaler.fit_transform(input_array)
    
    X = input_array_N
    y = output_array
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.3, 
        random_state=13
    )
    
    classifier = LogisticRegression(
        tol= 1e-5,
        C=0.1,
        class_weight = 'balanced',
        #solver='sag',
        max_iter=1500
    )
    
    classifier.fit(X_train,y_train)
    
    viz = plot_roc_curve(
        classifier, 
        X_test, 
        y_test,
        name='Test ROC'.format(0),
        alpha=0.5, lw=1, ax=ax
    )
    
    viz_train = plot_roc_curve(
        classifier, 
        X_train, 
        y_train,
        name='Train ROC'.format(1),
        alpha=0.5, lw=1, ax=ax
    ) 
    
    ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
        label='Chance', alpha=.8)
    
    ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
       title="Receiver operating characteristic example")
    ax.legend(loc="lower right")
    time_end=time.time()
    print('Training done, time cost:',time_end-time_start,'s')
    return classifier, X_test, y_test, X_train, y_train

In [ ]:
def train_log_cross_validation(data):
    #Get dummy value
    output_array = np.asarray(data['default_flag'].astype(int))
    input_array = np.c_[
        data[['CREDIT_SCORE',#0
              'ORIGINAL_DTI_RATIO',#1
              'ORIGINAL_UPB',#2
              'ORIGINAL_LTV',#3
              'ORIGINAL_LOAN_TERM',#4
              'ORIGINAL_INTEREST_RATE'#5
             ]],
        np.asarray(pd.get_dummies(data['FIRST_TIME_HOMEBUYER_FLAG'])),
        np.asarray(pd.get_dummies(data['NUMBER_OF_UNITS'])),
        np.asarray(pd.get_dummies(data['OCCUPANCY_STATUS'])),
        np.asarray(pd.get_dummies(data['CHANNEL'])),
        np.asarray(pd.get_dummies(data['PROPERTY_TYPE'])),
        np.asarray(pd.get_dummies(data['LOAN_PURPOSE'])),
        np.asarray(pd.get_dummies(data['NUMBER_OF_BORROWERS']))
    ]
    
    #Normalise
    min_max_scaler = preprocessing.MinMaxScaler()
    input_array_N = min_max_scaler.fit_transform(input_array)
    
    X = input_array_N
    y = output_array
    
    #devide Flods for cv
    cv = StratifiedKFold(n_splits=6)
    #define classifier
    classifier = LogisticRegression(
        solver='saga',
        max_iter=1500
    )

    #tpr lists and auc value list
    tprs = []
    aucs = []
    
    #For ploting, prepare 500 points from 0-1
    mean_fpr = np.linspace(0, 1, 500)
      
    #Loop training for every fold
    for i, (train, test) in enumerate(cv.split(X, y)):
        classifier.fit(X[train], y[train])
        # put the curve in ax through 'ax = ax'
        viz = plot_roc_curve(classifier, X[test], y[test],
                             name='ROC fold {}'.format(i),
                             alpha=0.3, lw=1, ax=ax)

        #Plot every point (500) form 0-1, similiar to bin
        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        #Buff the result 
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)

    #Plot chance
    ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
            label='Chance', alpha=.8)

    #mean value for each colomn
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    
    std_auc = np.std(aucs)
    #Plot mean
    ax.plot(mean_fpr, mean_tpr, color='b',
            label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
            lw=2, alpha=.8)

    #Plot standard tpr (Doesn't know the point for this step yet)
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                    label=r'$\pm$ 1 std. dev.')

    
    #Configure the diagram 
    ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
           title="Receiver operating characteristic example")
    ax.legend(loc="lower right")

In [ ]:
if __name__ == '__main__':
    
#     year_list = ['2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014']
    year_list = ['2004', '2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018']

#     for year_ in year_list:
#         data_path = 'data/{year}/data'.format(year=year_)
#         data_path_time = 'data/{year}/data_time'.format(year=year_)
#         data = load_data(data_path, data_path_time)
#         data.to_csv('data_flag/{}_flag.csv'.format(year_))
        
    for year_ in year_list:
        print("Loading data...")
        data_path = 'data_flag/{year}_flag.csv'.format(year=year_)  
        cols = pd.read_csv(data_path).columns
        data = pd.read_csv(data_path, usecols = cols[1:]) 
        y = np.asarray(data['default_flag'].astype(int))
        fig, ax = plt.subplots(figsize=(15, 8))         
        print("Training start...")
        model, X_test, y_test, X_train, y_train= train_log(data, fig, ax, year_)
        plt.show()
        fig.savefig('./log_AUC_without_norm/AUC_{year}_log.jpg'.format(year=year_)) 
        dump(model, './log_without_norm_models/{year}_logistic.joblib'.format(year=year_))
        np.savetxt("./without_norm_datasets/test/features/{year}_log.csv".format(year=year_), X_test, delimiter=",")
        np.savetxt("./without_norm_datasets/test/target/{year}_log.csv".format(year=year_), y_test, delimiter=",")
        np.savetxt("./without_norm_datasets/train/features/{year}_log.csv".format(year=year_), X_train, delimiter=",")
        np.savetxt("./without_norm_datasets/train/target/{year}_log.csv".format(year=year_), y_train, delimiter=",
         

In [ ]:
#     year_list = ['2004', '2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018']
#     data_ = []
#     default = []
#     total = []
#     for year_ in year_list:
#         print("Loading data...")
#         data_path = 'data_flag/{year}_flag.csv'.format(year=year_)  
#         cols = pd.read_csv(data_path).columns
#         data = pd.read_csv(data_path, usecols = cols[1:]) 
#         y = np.asarray(data['default_flag'].astype(int))
#         u_ele, ct_ele = np.unique(y, return_counts=True)
#         default.append(ct_ele[1])
#         total.append(np.size(y))
#     data_.append(default)
#     data_.append(total)
#     columns = ['%d' % x for x in np.arange(2004,2019)]
#     rows = ['default','number of accounts'] 
    

In [ ]:
#     fig, ax = plt.subplots(figsize=(30, 20))
#     values = np.arange(0, 2500, 500)
#     value_increment = 10000
#     colors = plt.cm.BuPu(np.linspace(0, 0.5, len(rows)))
#     n_rows = len(data_)

#     index = np.arange(len(columns)) + 0.6
#     bar_width = 0.4

#     # Initialize the vertical-offset for the stacked bar chart.
#     y_offset = np.zeros(len(columns))

#     # Plot bars and create text labels for the table
#     #cell_text = []
#     for row in range(n_rows):
#         bar = plt.bar(index, data_[row], bar_width, bottom=y_offset, color=colors[row])
#         y_offset = y_offset + data_[row]
#         cell_text.append(['%d' % x for x in y_offset])
#     # Reverse colors and text labels to display the last value at the top.
#     colors = colors[::1]
#     #cell_text.reverse()

#     # Add a table at the bottom of the axes
#     the_table = plt.table(cellText=data_,
#                           rowLabels=rows,
#                           rowColours=colors,
#                           colLabels=columns,
#                           loc='bottom')
#     the_table.set_fontsize(25)
#     the_table.scale(1.0, 2.0)  # may help

#     # Adjust layout to make room for the table:
#     #plt.subplots_adjust(left=0.2, bottom=0.2)
#     values = np.arange(0, 2000000, 500000)
#     value_increment = 1000
#     plt.ylabel("Person",fontsize = 20)
#     plt.yticks(values, ['%d' % val for val in values], fontsize = 20)
#     plt.xticks([])
#     plt.title('Default counts by years',fontsize = 20)

#     plt.show()

In [ ]:
# fig.savefig('./Defaults.jpg')